In [61]:
import scrublet as scr
import scipy.io
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import csv
import scanpy as sc


#gzip -dk outs_*/filtered_feature_bc_matrix/*.gz

In [90]:
samples = pd.read_csv ("../../../data/raw/scRNAseq/scRNASEQ_DATA/Pediatric_samples/Pediatric_sample_tracker.csv")
samples["Sanger Sample ID"][0]

'4918STDY7273964'

In [104]:
doublet_threshold = []

for samp in range(0, len(samples["Sanger Sample ID"])):
    input_dir = '../../../data/raw/scRNAseq/scRNASEQ_DATA/Pediatric_samples/pediatric_cellranger302_GRCh38-3.0.0/outs_'+samples["Sanger Sample ID"][samp]+'/filtered_feature_bc_matrix/'
    counts_matrix = scipy.io.mmread(input_dir + 'matrix.mtx').T.tocsc()
    genes = np.array(scr.load_genes(input_dir + 'features.tsv', delimiter='\t', column=1))
    
    print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
    print('Number of feature in feature list: {}'.format(len(genes)))
    
    scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
    
    doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)
    #save plot
    scrub.plot_histogram()
    plt.savefig('../../../figs/scRNAseq/primary/doublets/doublet_histogram_' + samples["Sanger Sample ID"][samp] + '.pdf')
    
    #save thesholds to report
    doublet_threshold.append(scrub.threshold_)
    
    #save output for R/seurat
    barcodes = pd.read_csv (input_dir + 'barcodes.tsv', sep = '\t', header=None)
    
    df = pd.DataFrame({
    'index': barcodes[0],
    'doublet_score': scrub.doublet_scores_obs_,
    'predicted_doublet': scrub.predicted_doublets_
    })
    df.to_csv('../../../output/scrublet_output_table_' + samples["Sanger Sample ID"][samp] + '.csv', index=False)

Counts matrix shape: 6459 rows, 33538 columns
Number of feature in feature list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.38
Detected doublet rate = 0.9%
Estimated detectable doublet fraction = 17.9%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 4.9%
Elapsed time: 5.3 seconds
Counts matrix shape: 4993 rows, 33538 columns
Number of feature in feature list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.56
Detected doublet rate = 0.2%
Estimated detectable doublet fraction = 7.0%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 2.6%
Elapsed time: 3.9 seconds
Counts matrix shape: 3582 rows, 33538 columns
Number of feature in feature list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet 

/home/redgar/anaconda3/envs/IBD_scRNAseq/lib/python3.8/site-packages/scrublet/scrublet.py:487: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axs = plt.subplots(1, 2, figsize = fig_size)


Counts matrix shape: 2466 rows, 33538 columns
Number of feature in feature list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.49
Detected doublet rate = 0.1%
Estimated detectable doublet fraction = 11.5%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 1.1%
Elapsed time: 1.5 seconds
Counts matrix shape: 1705 rows, 33538 columns
Number of feature in feature list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.45
Detected doublet rate = 0.1%
Estimated detectable doublet fraction = 10.9%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 1.1%
Elapsed time: 0.9 seconds
Counts matrix shape: 4114 rows, 33538 columns
Number of feature in feature list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet

In [107]:
doublet_threshold
doublet_threshold_df = pd.DataFrame(doublet_threshold)
doublet_threshold_df.to_csv("../../../output/doublet_threshold_primary.csv") 